# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [5]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np



In [6]:
df = pd.read_csv("http://www.ats.ucla.edu/stat/data/binary.csv")
df.head()

,admit,gre,gpa,rank
0,0,380,3.61,3
1,1,660,3.67,3
2,1,800,4.00,1
3,1,640,3.19,4
4,0,520,2.93,4


In [7]:
df.columns = ["admit", "gre", "gpa", "prestige"]
print df.columns

Index([u'admit', u'gre', u'gpa', u'prestige'], dtype='object')


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [14]:
# frequency table for prestige and whether or not someone was admitted
FT = pd.crosstab(index=df['admit'], columns=df['prestige'])
print FT

prestige   1   2   3   4
admit                   
0         28  97  93  55
1         33  54  28  12


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [15]:
dummy_prestige = pd.get_dummies(df['prestige'], prefix='prestige')
print dummy_prestige.head()

   prestige_1  prestige_2  prestige_3  prestige_4
0         0.0         0.0         1.0         0.0
1         0.0         0.0         1.0         0.0
2         1.0         0.0         0.0         0.0
3         0.0         0.0         0.0         1.0
4         0.0         0.0         0.0         1.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: We only use 3 of the 4 'prestige' dummy variables to avoid multicollinearity. 

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [16]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
#pandas.crosstab(index, columns, values=None, rownames=None, colnames=None, aggfunc=None, margins=False, dropna=True)

num_1_school = pd.crosstab(df['admit'], df['prestige_2'], rownames=['admit'])
num_1_school

KeyError: 'prestige_2'

#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

33/28

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [ ]:
94/245

#### 3.3 Calculate the odds ratio

(33)(245)/(28)(94)

#### 3.4 Write this finding in a sentenance: 

Answer: Someone who attended #1 ranked undergrad college is 3x more likely to be accepted to grad school

#### 3.5 Print the cross tab for prestige_4

In [ ]:
P4 = pd.crosstab(index=data['admit'], columns=data['prestige_4.0'])
print P4

#### 3.6 Calculate the OR 

(12)(216)/(55)(114)

#### 3.7 Write this finding in a sentence

Answer: A person attending a #4 ranked college has only an 18% chance of being accepted into grad school

## Part 4. Analysis

In [18]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
df2 = df[cols_to_keep].join(dummy_prestige.ix[:, 'prestige_1':])


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [19]:
# manually add the intercept
df2['intercept'] = 1

#### 4.1 Set the covariates to a variable called train_cols

In [20]:
train_cols = df2.columns[1:]

#### 4.2 Fit the model

In [21]:
logit = sm.Logit(df2['admit'], df2[train_cols])
result2 = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573147
         Iterations 6


#### 4.3 Print the summary results

In [22]:
print result2.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  400
Model:                          Logit   Df Residuals:                      394
Method:                           MLE   Df Model:                            5
Date:                Tue, 13 Dec 2016   Pseudo R-squ.:                 0.08292
Time:                        12:23:41   Log-Likelihood:                -229.26
converged:                       True   LL-Null:                       -249.99
                                        LLR p-value:                 7.578e-08
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
gre            0.0023      0.001      2.070      0.038         0.000     0.004
gpa            0.8040      0.332      2.423      0.015         0.154     1.454
prestige_1    -0.0846        nan        nan        n

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [23]:
params = result2.params
conf = result2.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                2.5%     97.5%        OR
gre         1.000120  1.004418  1.002267
gpa         1.166122  4.281877  2.234545
prestige_1       NaN       NaN  0.918904
prestige_2       NaN       NaN  0.467659
prestige_3       NaN       NaN  0.240562
prestige_4       NaN       NaN  0.194750
intercept        NaN       NaN  0.020133


#### 4.5 Interpret the OR of Prestige_2

Answer: Prestige 2 attendees are 46% less likely to be accepted to grad school

#### 4.6 Interpret the OR of GPA

Answer: Students with 4 or higher gpa are more than twice as likely to be accepted to grad school than students with 1 point gpa

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [24]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [26]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(df2['gre'].min(), df2['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(df2['gpa'].min(), df2['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
df3 = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


In [27]:
print df3.head()

       0         1    2    3
0  220.0  2.260000  1.0  1.0
1  220.0  2.260000  2.0  1.0
2  220.0  2.260000  3.0  1.0
3  220.0  2.260000  4.0  1.0
4  220.0  2.453333  1.0  1.0


#### 5.1 Recreate the dummy variables

In [28]:
# recreate the dummy variables
df3.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(df3['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']

# keep only what we need for making predictions


In [29]:
cols_to_keep = ['gre', 'gpa', 'intercept']
df3 = df3[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print df3.head()

     gre       gpa  intercept  prestige_1  prestige_2  prestige_3  prestige_4
0  220.0  2.260000        1.0         1.0         0.0         0.0         0.0
1  220.0  2.260000        1.0         0.0         1.0         0.0         0.0
2  220.0  2.260000        1.0         0.0         0.0         1.0         0.0
3  220.0  2.260000        1.0         0.0         0.0         0.0         1.0
4  220.0  2.453333        1.0         1.0         0.0         0.0         0.0


#### 5.2 Make predictions on the enumerated dataset

In [31]:
df3['admit_pred'] = result2.predict(df3[train_cols])
print df3.head()


     gre       gpa  intercept  prestige_1  prestige_2  prestige_3  prestige_4  \
0  220.0  2.260000        1.0         1.0         0.0         0.0         0.0   
1  220.0  2.260000        1.0         0.0         1.0         0.0         0.0   
2  220.0  2.260000        1.0         0.0         0.0         1.0         0.0   
3  220.0  2.260000        1.0         0.0         0.0         0.0         1.0   
4  220.0  2.453333        1.0         1.0         0.0         0.0         0.0   

   admit_pred  
0    0.157801  
1    0.087056  
2    0.046758  
3    0.038194  
4    0.179574  


In [32]:
print df3.tail()

       gre       gpa  intercept  prestige_1  prestige_2  prestige_3  \
395  800.0  3.806667        1.0         0.0         0.0         0.0   
396  800.0  4.000000        1.0         1.0         0.0         0.0   
397  800.0  4.000000        1.0         0.0         1.0         0.0   
398  800.0  4.000000        1.0         0.0         0.0         1.0   
399  800.0  4.000000        1.0         0.0         0.0         0.0   

     prestige_4  admit_pred  
395         1.0    0.338676  
396         0.0    0.738408  
397         0.0    0.589590  
398         0.0    0.424948  
399         1.0    0.374314  


#### 5.3 Interpret findings for the last 4 observations

Answer: 
+ 396 = perfect gre and gpa, attended #1 undergrad uni, has 74% chance of being accepted
+ 397 = perfect gre and gpa, attended #2 undergrad uni, has 59% chance of being accepted
+ 398 = perfect gre and gpa, attended #3 undergrad uni, has 43% chance of being accepted
+ 399 = perfect gre and gpa, attended #4 undergrad uni, has 37% chance of being accepted


## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.